<a href="https://colab.research.google.com/github/livio-24/test-deduplication/blob/main/sbert_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# If using Google Drive, mount it
from google.colab import drive
drive.mount('/content/drive')  # Only if dataset is stored in Google Drive


Mounted at /content/drive


In [2]:
# ========================
# 🔥 1️⃣ SETUP & INSTALL DEPENDENCIES
# ========================

# Install required libraries
!pip install -q sentence-transformers pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.0 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

In [5]:
# ========================
# 📂 2️⃣ LOAD DATASET
# ========================

# Load dataset (modify path if needed)
file_path = "/content/drive/MyDrive/tesi/val_set2.csv"  # Adjust to your file location
df = pd.read_csv(file_path)
#df = df[:100]

In [6]:
df

,id,url,title,description,text,date_publication,time_publication,newspaper,date_event,tags,is_dup
0,1,https://notizie.virgilio.it/intossicazione-ali...,Intossicazione alimentare nelle Rsa di Firenze...,Allarme intossicazione alimentare nelle Rsa di...,Allarme intossicazione alimentare nella Rsa Vi...,2025-02-13,17:11:44,Virgilio Notizie,2025-02-13,"{FIRENZE,TOSCANA}",0
1,2,https://notizie.virgilio.it/neonata-trovata-mo...,"Neonata trovata morta in culla a Lucca, la mad...",Neonata trovata morta a Lucca nella sua culla:...,Proseguono le indagini sul caso della neonata ...,2025-02-12,11:56:37,Virgilio Notizie,2025-02-07,"{LUCCA,TOSCANA}",0
2,3,https://notizie.virgilio.it/studente-si-accasc...,Studente si accascia a scuola a Cremona durant...,Uno studente di 14 anni è morto dopo aver accu...,Si è accasciato durante l’ora di ginnastica ed...,2025-02-10,19:10:22,Virgilio Notizie,2025-02-10,"{CREMONA,LOMBARDIA}",0
3,4,https://notizie.virgilio.it/omicidio-baby-sitt...,Omicidio baby sitter Jhoanna Nataly Quintanill...,Continuano le indagini sull'omicidio della bab...,Ha ucciso la compagna Jhoanna Nataly Quintanil...,2025-02-10,18:14:14,Virgilio Notizie,2025-02-09,{},1
4,5,https://notizie.virgilio.it/nuova-perizia-psic...,Nuova perizia psichiatrica per Alessia Pifferi...,La Corte d'Assise d'Appello ha disposto una nu...,Nuova perizia psichiatrica. È la decisione dei...,2025-02-10,16:55:31,Virgilio Notizie,2025-02-10,"{""ALESSIA PIFFERI""}",1
...,...,...,...,...,...,...,...,...,...,...,...
95,96,https://notizie.virgilio.it/rapina-nella-villa...,Rapina nella villa di Maria Sole Agnelli a Rom...,Alcuni ladri hanno fatto irruzione nella casa ...,Una banda di rapinatori ha fatto irruzione nel...,2025-01-09,19:01:31,Virgilio Notizie,2025-01-09,"{LAZIO,ROMA}",0
96,97,https://notizie.virgilio.it/omicidio-di-kelly-...,"Omicidio di Kelly Egbon a Schio, 32enne morto ...","Confessato l'omicidio di Kelly Egbon, il 32enn...",Queen Enabuele ha confessato l’omicidio del fi...,2025-01-09,10:09:32,Virgilio Notizie,NaN,"{VENETO,VICENZA}",0
97,98,https://notizie.virgilio.it/neonato-trovato-mo...,"Neonato trovato morto a Bari in culla, autopsi...",L'autopsia del neonato trovato morto nella cul...,Il neonato trovato morto a Bari lo scorso 2 ge...,2025-01-08,23:46:17,Virgilio Notizie,2025-01-02,"{BARI,PUGLIA}",0
98,99,https://notizie.virgilio.it/giuseppe-barbaro-m...,Giuseppe Barbaro morto in ospedale a Palermo d...,"Giuseppe Barbaro, 76 anni, è morto all'ospedal...","Giuseppe Barbaro, 76 anni, è morto dopo 17 gio...",2025-01-08,17:54:05,Virgilio Notizie,2024-12-24,"{PALERMO,SICILIA}",0


# TEST 2 - embedding per title, description e text. Calcolo valore di similarità combinando i vari embedding

In [35]:
!python -m spacy download it_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 40.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [33]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import spacy
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm


In [39]:
import torch

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [36]:
# Carica il modello spaCy per l'italiano
nlp = spacy.load("it_core_news_sm")

# Carica il modello SBERT (Multilingue)
#model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
model = SentenceTransformer('nickprock/sentence-bert-base-italian-uncased').to(device)

# Funzione per ottenere la media delle embedding delle frasi
def get_average_sentence_embedding(text, model):
    # Tokenizza il testo in frasi usando spaCy
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    # Calcola le embedding per ogni frase
    sentence_embeddings = model.encode(sentences,batch_size=32 , convert_to_numpy=True)# show_progress_bar=True)

    # Calcola la media delle embedding delle frasi
    avg_embedding = np.mean(sentence_embeddings, axis=0)
    return avg_embedding

# Calcolare le embedding per titolo, descrizione e testo completo
title_embeddings = model.encode(df["title"].tolist(),batch_size=32 , convert_to_numpy=True, show_progress_bar=True)
description_embeddings = model.encode(df["description"].tolist(), batch_size=32 , convert_to_numpy=True, show_progress_bar=True)

# Per il testo completo, calcoliamo la media delle embedding delle frasi
full_text_embeddings = np.array([get_average_sentence_embedding(text, model) for text in tqdm(df["text"])])

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 100/100 [04:42<00:00,  2.83s/it]


In [ ]:
#np.save("/content/drive/MyDrive/test-sbert-it/title_embeddings.npy", title_embeddings)
#np.save("/content/drive/MyDrive/test-sbert-it/description_embeddings.npy", description_embeddings)
#np.save("/content/drive/MyDrive/test-sbert-it/text_embeddings.npy", full_text_embeddings)

In [40]:
title_embeddings = np.load("/content/drive/MyDrive/test-sbert-it/title_embeddings.npy")
description_embeddings = np.load("/content/drive/MyDrive/test-sbert-it/description_embeddings.npy")
full_text_embeddings = np.load("/content/drive/MyDrive/test-sbert-it/text_embeddings.npy")

In [224]:
# Calcolare la similarità coseno usando sklearn
title_sim_matrix = cosine_similarity(title_embeddings)
description_sim_matrix = cosine_similarity(description_embeddings)
full_text_sim_matrix = cosine_similarity(full_text_embeddings)

x = 1
y = 1
z = 2
# Combinare i valori di similarità (media ponderata o somma)
combined_sim_matrix = (x * title_sim_matrix + y * description_sim_matrix + z * full_text_sim_matrix) / (x + y + z)


In [265]:
# Impostare una soglia per trovare le coppie simili
SIMILARITY_THRESHOLD = 0.72# Adatta a seconda dei tuoi dati

# Estrai le coppie simili che superano la soglia
near_duplicates = []
N = len(df)

for i in range(N):
    for j in range(i + 1, N):  # Escludiamo le coppie duplicate e se stessa
        #date_i = pd.to_datetime(df.iloc[i]["date_publication"])
        #date_j = pd.to_datetime(df.iloc[j]["date_publication"])

        # Controlliamo che la differenza sia entro 5 giorni
        #if abs((date_i - date_j).days) <= 5:
          if combined_sim_matrix[i, j] > SIMILARITY_THRESHOLD:
            near_duplicates.append({
                "Title 1": df.iloc[i]["title"],
                "Title 2": df.iloc[j]["title"],
                "Combined Similarity": round(combined_sim_matrix[i, j], 3)
            })

# Visualizza i risultati
duplicates_df = pd.DataFrame(near_duplicates)
print("\n🔍 Sample Near-Duplicate Articles:")

# Salva i risultati (facoltativo)
duplicates_df.to_csv("/content/near_duplicate_combined_articles.csv", index=False)
print("\n✅ Near-duplicate detection completed! Results saved as 'near_duplicate_combined_articles.csv'")


🔍 Sample Near-Duplicate Articles:

✅ Near-duplicate detection completed! Results saved as 'near_duplicate_combined_articles.csv'


In [266]:
duplicates_df

,Title 1,Title 2,Combined Similarity
0,Omicidio baby sitter Jhoanna Nataly Quintanill...,Babysitter Jhoanna Nataly Quintanilla uccisa i...,0.791
1,"Incendio a Roma Labaro, donna anziana morta in...","Incendio a Oriolo Romano vicino Viterbo, donna...",0.759
2,Femminicidio e tentato suicidio a Venaria Real...,Ergastolo a Stefania Russolillo per la morte d...,0.744
3,Femminicidio e tentato suicidio a Venaria Real...,Ergastolo scampato per Salvatore Montefusco do...,0.774
4,Babysitter Jhoanna Nataly Quintanilla uccisa i...,Pablo Gonzalez confessa l'omicidio della compa...,0.786
5,Babysitter Jhoanna Nataly Quintanilla uccisa i...,"Cosa sappiamo su Jhoanna, la baby sitter scomp...",0.730
6,Morto uno dei ladri colpito dalla guardia giur...,Guardia giurata sorprende i ladri in casa del ...,0.794
7,"Uccisi in strada a Miano a Napoli, chi erano S...","Spari in strada a Napoli nel quartiere Miano, ...",0.794
8,Victor Ulinici condannato per la bici che ha c...,"Sara Cherici e la bici lanciata dai Murazzi, c...",0.789
9,Arrestato il figlio dell'uomo trovato morto a ...,Alex Cotoia assolto in Appello a Torino per l'...,0.755


In [267]:
#val_set = pd.read_csv("/content/drive/MyDrive/tesi/validation_set_duplication.csv")

In [268]:
grouped_df = duplicates_df.groupby('Title 1')['Title 2'].apply(list).reset_index()
grouped_df

,Title 1,Title 2
0,"Alex Cotoia assolto per l'omicidio del padre, ...",[Alex Cotoia assolto in Appello a Torino per l...
1,"Andrea Prospero trovato morto a Perugia, i sos...","[Andrea Prospero trovato morto a Perugia, stud..."
2,Arrestato il figlio dell'uomo trovato morto a ...,[Alex Cotoia assolto in Appello a Torino per l...
3,Babysitter Jhoanna Nataly Quintanilla uccisa i...,[Pablo Gonzalez confessa l'omicidio della comp...
4,"Cani azzannano e uccidono una donna a Latina, ...",[Donna morta azzannata dai cani a Latina nel g...
5,"Duplice femminicidio ma niente ergastolo, sent...",[Ergastolo scampato per Salvatore Montefusco d...
6,Ergastolo a Stefania Russolillo per la morte d...,[Chiesto l'ergastolo per l'omicidio della mamm...
7,Ergastolo scampato per Salvatore Montefusco do...,"[Salvatore Montefusco evita l'ergastolo, doppi..."
8,Femminicidio e tentato suicidio a Venaria Real...,[Ergastolo a Stefania Russolillo per la morte ...
9,"Incendio a Roma Labaro, donna anziana morta in...","[Incendio a Oriolo Romano vicino Viterbo, donn..."


In [269]:
df['dup_pred'] = 0  # Initialize with a default value

# Iterate through the grouped DataFrame
for index, row in grouped_df.iterrows():
    doc1_index = row['Title 1']

    df.loc[df['title'] == doc1_index, 'dup_pred'] = 1  # Convert the list to a string for storage

In [270]:
#val_set.to_csv("/content/drive/MyDrive/tesi/validation_set_duplication.csv", index=False)

In [271]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [272]:
y_true = df["is_dup"]
y_pred = df["dup_pred"]

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.86
Precision: 0.74
Recall: 0.61
F1 Score: 0.67
